# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Path('../backend/shuscribe').resolve()
sys.path.insert(0, str(Path('../backend').resolve()))

In [2]:
# Import necessary modules
import asyncio
from dotenv import load_dotenv
import os
from shuscribe.services.llm.session import LLMSession
from shuscribe.services.llm.providers.provider import (
    Message, GenerationConfig
)
from IPython.display import Markdown

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

TEST_MODELS ={
    "openai": "gpt-4o-mini",
    "anthropic": "claude-3-5-haiku-20241022",
    "gemini": "gemini-2.0-flash-001"
}

TEST_THINKING_MODELS = {
    "openai": "o3-mini-2025-01-31",
    "anthropic": "claude-3-7-sonnet-20250219",
    "gemini": "gemini-2.0-flash-thinking-exp"
}

STORY_DIR = Path("../backend/tests/resources/pokemon_amber")

# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

In [3]:
USE_CACHED_RESPONSES = False

In [4]:
# Streaming response
from typing import Type
from shuscribe.schemas.base import BaseOutputSchema
from shuscribe.schemas.llm import ThinkingConfig
from shuscribe.schemas.streaming import StreamChunk
from shuscribe.services.llm.streaming import StreamStatus


async def stream(
    provider_name: str, 
    model: str,
    messages: list[Message], 
    response_schema: Type[BaseOutputSchema] | None = None, 
    max_tokens: int | None = None,
    temp: float | None = None,
    thinking_config: ThinkingConfig | None = None,
    config: GenerationConfig | None = None
    ) -> StreamChunk | None:
    async with LLMSession.session_scope() as session:
        # Create a streaming config
        if not config:
            config = GenerationConfig(
                temperature=temp or 0.7,
                response_schema=response_schema if response_schema else None,
                max_output_tokens=max_tokens,
                thinking_config=thinking_config
            )
        
        print(f"{model if model else config.model}:")

        async for chunk in session.generate_stream(
            messages=messages,
            provider_name=provider_name,
            model=model,
            config=config
        ):
            print(chunk.text, end="", flush=True)

    if chunk:
        if chunk.status in (StreamStatus.COMPLETE, StreamStatus.ERROR):
            return chunk
    else:
        return None


In [5]:
import yaml

from shuscribe.schemas.pipeline import Chapter, StoryMetadata

with open(STORY_DIR / "story" / "_meta.yaml", "r") as f:
    meta = yaml.safe_load(f)
    STORY_METADATA = StoryMetadata(
        title=meta.get('story_title'),
        description=meta.get('story_description'),
        genres=meta.get('genres'),
        additional_tags=meta.get('additional_tags')
    )

CHAPTERS = []
for chapter in meta.get('chapters'):
    with open(STORY_DIR / "story" / chapter, "r") as f:
        chapter_id = chapter.split('.')[0]
        try:
            chapter_content = yaml.safe_load(f)
            CHAPTERS.append(Chapter(id=chapter_id, title=chapter_content.get('title'), content=chapter_content.get('content')))
        except Exception:
            continue



# Summarization Pipeline

## Chapter [1]

In [6]:
# Set Provider Name
from shuscribe.schemas.wikigen.entity import UpsertEntitiesOutSchema
from shuscribe.schemas.wikigen.wiki import WikiPage
from shuscribe.schemas.wikigen.summary import ChapterSummary


PROVIDER_NAME = "gemini"
CHAPTER_INDEX = 1

with open(STORY_DIR / f"{CHAPTER_INDEX-1}out" / "comprehensive_wiki.yaml", "r") as f:
    COMPREHENSIVE_WIKI = WikiPage.from_wiki_content("Comprehensive Wiki Page", f.read())

with open(STORY_DIR / f"{CHAPTER_INDEX-1}out" / "chapter_summary.yaml", "r") as f:
    PREV_SUMMARY = ChapterSummary.from_chapter_summary(CHAPTER_INDEX-1, f.read())

with open(STORY_DIR / f"{CHAPTER_INDEX-1}out" / "upsert_entities.yaml", "r") as f:
    UPSERT_ENTITIES = UpsertEntitiesOutSchema.model_validate_json(f.read())
# USE_CACHED_RESPONSES = False

### Chapter Summary

In [ ]:
from shuscribe.schemas.wikigen.summary import ChapterSummary
from shuscribe.services.llm.prompts import templates

templates.chapter.summary.reload()

summary_messages: list[Message] = templates.chapter.summary.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    story_metadata=STORY_METADATA,
    summary_so_far=COMPREHENSIVE_WIKI,
    recent_summaries=[PREV_SUMMARY],
)

# print(messages[-1].content)

summary_response = None

if USE_CACHED_RESPONSES:
    # if exists, load from file
    if (STORY_DIR / f"{CHAPTER_INDEX}out" / "chapter_summary.yaml").exists():
        with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "chapter_summary.yaml", "r") as f:
            summary_response = StreamChunk.model_validate_json(f.read())

# if we don't have a cached response, generate one
if not summary_response:
    summary_response = await run_async(stream(
        "", 
        "", 
        summary_messages, 
        config=templates.chapter.summary.default_config
        ))

print(summary_response.usage)
os.makedirs(STORY_DIR / f"{CHAPTER_INDEX}out", exist_ok=True)
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "chapter_summary.yaml", "w") as f:
    f.write(summary_response.model_dump_json(indent=2))

chapter_summary = ChapterSummary.from_chapter_summary(CHAPTER_INDEX, summary_response.accumulated_text)


gemini-2.0-flash-001:
<|STARTOFSUMMARY|>

## The Sleepless Night

*   "Amber" (Alexa) struggles to sleep, haunted by the events of Mewtwo's escape and her new reality. [!CHARACTER]
    *   She's disturbed by the feeling of being trapped in a ten-year-old's body, grappling with the physical and mental disconnect. [!CHARACTER]
    *   She reflects on Dr. Fuji's actions and questions her own right to exist in Amber's place. [!CHARACTER] [!THEME]
    *   She recalls her past life, including her mother, her cat, and her passion for Pokémon, feeling a sense of distance and wrongness. [!CHARACTER] [!CALLBACK]
*   She fears Team Rocket's potential pursuit and their possible plans for Mewtwo, as well as what they might do if they knew about her meta knowledge. [!FORESHADOW]
    *   She considers the different possible futures based on the anime, games, and manga. [!ALLUSION]
*   She feels like an anomaly, something not even Arceus planned for, and faces the prospect of continuing to pretend in 

Database module not implemented. Skipping save.


prompt_tokens=5096 completion_tokens=653


### Extract Entities

In [8]:
print(chapter_summary.to_prompt())

<Content>


## The Sleepless Night

*   "Amber" (Alexa) struggles to sleep, haunted by the events of Mewtwo's escape and her new reality. [!CHARACTER]
    *   She's disturbed by the feeling of being trapped in a ten-year-old's body, grappling with the physical and mental disconnect. [!CHARACTER]
    *   She reflects on Dr. Fuji's actions and questions her own right to exist in Amber's place. [!CHARACTER] [!THEME]
    *   She recalls her past life, including her mother, her cat, and her passion for Pokémon, feeling a sense of distance and wrongness. [!CHARACTER] [!CALLBACK]
*   She fears Team Rocket's potential pursuit and their possible plans for Mewtwo, as well as what they might do if they knew about her meta knowledge. [!FORESHADOW]
    *   She considers the different possible futures based on the anime, games, and manga. [!ALLUSION]
*   She feels like an anomaly, something not even Arceus planned for, and faces the prospect of continuing to pretend in this new reality. [!CHARACTER]

In [9]:
from shuscribe.schemas.wikigen.entity import ExtractEntitiesOutSchema


templates.entity.extract.reload()
extract_messages: list[Message] = templates.entity.extract.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    # current_chapter=Chapter(id=1, title="Chapter 1", content="This is a test chapter."),
    story_metadata=STORY_METADATA,
    chapter_summary=chapter_summary,
    summary_so_far=COMPREHENSIVE_WIKI,
    recent_summaries=[PREV_SUMMARY],
)

# print(extract_messages[-1].content)
extract_response = None
if USE_CACHED_RESPONSES:
    # if exists, load from file
    if (STORY_DIR / f"{CHAPTER_INDEX}out" / "extract_entities.yaml").exists():
        with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "extract_entities.yaml", "r") as f:
            extract_response = StreamChunk.model_validate_json(f.read())
            
# if we don't have a cached response, generate one
if not extract_response:
    extract_response = await run_async(stream(
        "", 
        "", 
        extract_messages, 
        config=templates.entity.extract.default_config
        ))
    
os.makedirs(STORY_DIR / f"{CHAPTER_INDEX}out", exist_ok=True)
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "extract_entities.yaml", "w") as f:
    f.write(extract_response.model_dump_json(indent=2))

extracted_entities = ExtractEntitiesOutSchema.model_validate_json(extract_response.accumulated_text)
print(extract_response.usage)


gpt-4o-mini:
{"entities":[{"description":"A ten-year-old girl who is the protagonist of the story, originally named Alexa. She is a reincarnation of Dr. Fuji's deceased daughter Amber, navigating her new life in the Pokémon world while grappling with her identity and memories from her past life.","narrative_role":"Protagonist struggling with her new identity and the consequences of her reincarnation.","significance_level":"Central","entity_type":"Character","identifier":"Amber (the Protagonist)","aliases":["Alexa"],"related_entities":["Dr. Fuji (the Scientist)","Mewtwo (the Legendary Pokémon)"]},{"description":"A middle-aged scientist who created the protagonist as a clone of his deceased daughter, Amber. He is emotionally attached to her and prioritizes her safety amidst the chaos following Mewtwo's escape.","narrative_role":"Amber's protector and creator, representing the emotional stakes of her existence.","significance_level":"Major","entity_type":"Character","identifier":"Dr. Fuji

Database module not implemented. Skipping save.


prompt_tokens=5941 completion_tokens=1018


### Upsert Entities and Relationships

In [10]:
from shuscribe.schemas.wikigen.entity import EntitySigLvl

ent_list = extracted_entities.filter_entities(EntitySigLvl.RELEVANT)
print(len(ent_list))
for entity in ent_list:
    print(entity.identifier)


9
Amber (the Protagonist)
Dr. Fuji (the Scientist)
Mewtwo (the Legendary Pokémon)
Dr. Fuji's Laboratory
Celadon Express
Cinnabar Island
Team Rocket
Identity and Existence
Possible Futures


In [11]:
# TODO
# manually define the entities to find...

for i, entity in enumerate(UPSERT_ENTITIES.entities):
    print(f"{i}: {entity.identifier}")

0: Mewtwo (the Legendary Pokémon)
1: Dr. Fuji (the Scientist)
2: Amber (the Protagonist)
3: Dr. Fuji's Laboratory
4: Shadow Mewtwo Raid Event
5: Identity Crisis (Pokemon: Ambertwo)
6: Laboratory Chaos (Pokemon: Ambertwo)
7: Pokémon Game (Pokemon: Ambertwo)


In [12]:
INDEXES = [0, 1, 2, 3, 4, 5]
existing_entities = [UPSERT_ENTITIES.entities[i].model_dump() for i in INDEXES]
type(existing_entities[0])

dict

In [13]:
# for batch in extracted_entities.batch_for_upsert(EntitySigLvl.RELEVANT):
#     for entity in batch:
#         print(entity)


In [14]:
from shuscribe.schemas.provider import LLMUsage
from shuscribe.schemas.wikigen.entity import UpsertEntitiesOutSchema
from shuscribe.schemas.wikigen.entity import EntitySigLvl

upsert_entities = UpsertEntitiesOutSchema(entities=[])

if USE_CACHED_RESPONSES:
    # if exists, load from file
    if (STORY_DIR / f"{CHAPTER_INDEX}out" / "upsert_entities.yaml").exists():
        with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "upsert_entities.yaml", "r") as f:
            upsert_entities = UpsertEntitiesOutSchema.model_validate_json(f.read())

if len(upsert_entities.entities) == 0:
    upsert_entities = UpsertEntitiesOutSchema(entities=[])
    total_usage = LLMUsage(prompt_tokens=0, completion_tokens=0)
    templates.entity.upsert.reload()        

    for batch in extracted_entities.batch_for_upsert(EntitySigLvl.RELEVANT, chunk_size=5):
        upsert_messages: list[Message] = templates.entity.upsert.format( 
            current_chapter=CHAPTERS[CHAPTER_INDEX],
            entity_batch=batch,
            story_metadata=STORY_METADATA,
            chapter_summary=chapter_summary,
            
            existing_entities=existing_entities,
            summary_so_far=COMPREHENSIVE_WIKI,
            recent_summaries=[PREV_SUMMARY],
        )

        upsert_response = await run_async(stream(
            "", 
            "", 
            upsert_messages, 
            config=templates.entity.upsert.default_config
            ))
        upsert_entities.entities.extend(UpsertEntitiesOutSchema.model_validate_json(upsert_response.accumulated_text).entities)
        total_usage.prompt_tokens += upsert_response.usage.prompt_tokens
        total_usage.completion_tokens += upsert_response.usage.completion_tokens
        print(upsert_response.usage)

os.makedirs(STORY_DIR / f"{CHAPTER_INDEX}out", exist_ok=True)
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "upsert_entities.yaml", "w") as f:
    f.write(upsert_entities.model_dump_json(indent=2))
    
print(total_usage)

gemini-2.0-flash-001:
{
  "entities": [
    {
      "old_identifier": "Amber (the Protagonist)",
      "identifier": "Amber (the Protagonist)",
      "detailed_description": "*   The protagonist, originally named Alexa, who died after being hit by a truck.\n*   Reincarnated in the body of a ten-year-old girl named Amber.\n*   Awakens in a tank filled with amber fluid.\n*   Experiences a sense of displacement and confusion regarding her new body and voice.\n*   Displays a caring nature, expressing concern for Dr. Fuji's well-being.\n*   Retains memories of her past life as Alexa, including knowledge of Pokémon and game mechanics.\n*   Struggles with insomnia, fearing that sleep will sever her connection to her former self.\n*   Feels guilt and questions her right to exist in Amber's place.\n*   Experiences a disconnect between her adult thoughts and her child's body.",
      "narrative_role": "Protagonist who drives the story forward through her actions and decisions, dealing with theme

Database module not implemented. Skipping save.


prompt_tokens=9330 completion_tokens=2246
gemini-2.0-flash-001:
{
  "entities": [
    {
      "old_identifier": null,
      "identifier": "Cinnabar Island",
      "detailed_description": "*   A volcanic island featuring a beach where the Celadon Express Pidgeot flight service operates.\n*   The island's volcano looms in the background, contrasting with the advanced technology hidden beneath it.\n*   Represents the vibrant and adventurous world of Pokémon, a stark contrast to the sterile laboratory environment.\n*   The island's natural beauty symbolizes the freedom and exploration Amber is about to experience.",
      "narrative_role": "Symbolizes the adventure and exploration Amber is about to undertake, representing the vibrant world of Pokémon and contrasting with the sterile laboratory environment.",
      "facts": [],
      "removed_facts": [],
      "entity_types": [
        "Location"
      ],
      "aliases": [],
      "related_entities": []
    },
    {
      "old_identifier":

Database module not implemented. Skipping save.


prompt_tokens=9142 completion_tokens=1130
prompt_tokens=18472 completion_tokens=3376


In [15]:
# print(upsert_entities.model_dump_json(indent=2))

### Story So Far Summary

In [16]:

from shuscribe.schemas.wikigen.story import WikiPage


templates.story.comprehensive_wiki.reload()
comprehensive_wiki_messages: list[Message] = templates.story.comprehensive_wiki.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    chapter_summary=chapter_summary,
    key_entities=upsert_entities,
    
    story_metadata=STORY_METADATA,
    # summary_so_far=summary_so_far,
    # recent_summaries=recent_summaries_prompt,
)

# print(comprehensive_summary_messages[-1].content)
comprehensive_wiki_response = None
if USE_CACHED_RESPONSES:
    # if exists, load from file
    if (STORY_DIR / f"{CHAPTER_INDEX}out" / "comprehensive_wiki.yaml").exists():
        with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "comprehensive_wiki.yaml", "r") as f:
            comprehensive_wiki_response = StreamChunk.model_validate_json(f.read())

if not comprehensive_wiki_response:
    comprehensive_wiki_response = await run_async(stream(
        "", 
        "", 
        comprehensive_wiki_messages, 
        config=templates.story.comprehensive_wiki.default_config
        ))

os.makedirs(STORY_DIR / f"{CHAPTER_INDEX}out", exist_ok=True)
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "comprehensive_wiki.yaml", "w") as f:
    f.write(comprehensive_wiki_response.model_dump_json(indent=2))

print(comprehensive_wiki_response.usage)
comprehensive_wiki = WikiPage.from_wiki_content("Comprehensive Wiki Page", comprehensive_wiki_response.accumulated_text)


claude-3-7-sonnet-20250219:
<ANTHROPIC_THINKING>I need to update the comprehensive wiki document by integrating the new information from Chapter 2. Let me first understand what's in the latest chapter and then find a way to integrate it into the existing wiki.

From the new chapter, we learn:
1. Amber struggles with insomnia and existential questions about her identity
2. She fears Team Rocket might pursue her for her meta-knowledge
3. She reflects on her past life as Alexa
4. She and Dr. Fuji travel via the Celadon Express, a Pidgeot-based flight service
5. She experiences the reality of Pokémon for the first time, being awestruck by the Pidgeot
6. The journey begins toward Celadon City

I need to integrate this information into the wiki document, ensuring it flows naturally with the existing content. I should organize it under appropriate sections rather than simply adding a new "Chapter 2" section.

I'll create/update sections related to:
- The Protagonist (Amber/Alexa)
- Locations 

Database module not implemented. Skipping save.


prompt_tokens=8016 completion_tokens=1994


In [17]:
# display markdown
Markdown(comprehensive_wiki.content)




# Pokémon: Ambertwo

## World Overview

The world of Pokémon: Ambertwo represents a complex blend of familiar Pokémon media elements with unique twists. This universe contains elements recognizable from games, anime, and manga, creating a rich tapestry where genetically engineered Pokémon exist alongside traditional training methods. The story unfolds primarily on [[Cinnabar Island]] before beginning to expand to other regions.

### Key Locations

#### [[Dr. Fuji's Laboratory]]

Hidden beneath [[Cinnabar Island]], this advanced research facility served as the birthplace of [[Mewtwo (the Legendary Pokémon)]] and other genetic experiments. Featuring amber-fluid-filled tanks, an evolution acceleration chamber, and other cutting-edge technology, the laboratory represented the pinnacle of forbidden Pokémon science. During [[Mewtwo (the Legendary Pokémon)]]'s violent awakening, much of the facility was destroyed, with tanks shattered and structural damage throughout.

#### [[Cinnabar Island]]

A volcanic island known for its tourism and beaches, [[Cinnabar Island]] hides secrets beneath its picturesque surface. The island's volcano looms over the landscape, creating a striking visual contrast to the modern facilities hidden below. The black volcanic sand beaches serve as a departure point for the [[Celadon Express]] flying service, connecting the island to mainland destinations.

## Key Characters

### [[Amber (the Protagonist)]]

Originally a Pokémon fan named Alexa from another world, the protagonist was reincarnated in the body of ten-year-old Amber following a fatal accident involving a truck. She awakened in a clone tank in [[Dr. Fuji's Laboratory]], quickly realizing she possessed the body of Dr. Fuji's deceased daughter.

Amber struggles with the disconnect between her adult consciousness and her child's body, experiencing significant dysphoria and questioning her right to exist in Amber's place. She maintains memories of her past life—including detailed knowledge of Pokémon games, anime, and manga—which gives her unique meta-knowledge about this world.

Following [[Mewtwo (the Legendary Pokémon)]]'s escape, Amber suffers from insomnia, fearing that sleep might sever her connection to her former identity. Her guilt about occupying Amber's body mingles with anxiety about [[Team Rocket]]'s potential interest in her, especially if they discovered her extensive knowledge of possible futures.

### [[Dr. Fuji (the Scientist)]]

A brilliant but morally compromised scientist who leads genetic experimentation on [[Cinnabar Island]]. Dr. Fuji created [[Mewtwo (the Legendary Pokémon)]] and attempted to resurrect his deceased daughter through cloning. His protective attitude toward [[Amber (the Protagonist)]] suggests he believes the cloning was successful, unaware that her consciousness belongs to someone else entirely.

Following the laboratory incident, Dr. Fuji sustained injuries including welts and a shoulder gash but remained primarily focused on Amber's safety. He accompanies her on the [[Celadon Express]], continuing his role as her guardian.

### [[Mewtwo (the Legendary Pokémon)]]

A genetically engineered Psychic-type Pokémon of immense power, [[Mewtwo (the Legendary Pokémon)]] represents the pinnacle of [[Dr. Fuji's Laboratory]]'s work. Created to be the world's strongest Pokémon, Mewtwo awakened prematurely and violently, destroying much of the laboratory during its escape.

The clone's raw power exists outside the natural order, protected by psychic shields though still vulnerable to physical attacks. During its escape, Mewtwo briefly made eye contact with [[Amber (the Protagonist)]], showing a flicker of recognition before fleeing.

Following its escape, Mewtwo is somewhere in the wild, grappling with its own existence and purpose—a parallel to Amber's own existential crisis.

## Organizations

### [[Team Rocket]]

A criminal organization involved in unethical Pokémon experiments and exploitation. [[Team Rocket]] appears to have funded [[Dr. Fuji's Laboratory]] and lost both their investment and their prize weapon when [[Mewtwo (the Legendary Pokémon)]] escaped.

[[Amber (the Protagonist)]] fears their potential interest in her, especially given her meta-knowledge of their operations and plans from various Pokémon media timelines. She contemplates different possible futures for Mewtwo based on her knowledge of the anime, games, and manga, including scenarios where [[Team Rocket]] recaptures Mewtwo for Giovanni's ambitions.

## Events

### The Escape

[[Mewtwo (the Legendary Pokémon)]]'s violent awakening and subsequent escape from [[Dr. Fuji's Laboratory]] marks the inciting incident of the story. During this chaos, [[Amber (the Protagonist)]] witnessed Mewtwo's raw power firsthand, including its psychic abilities and vulnerability to physical attacks from an Arcanine. The laboratory sustained significant damage, forcing [[Dr. Fuji (the Scientist)]] and Amber to flee.

### The Journey Begins

Following the laboratory incident, [[Amber (the Protagonist)]] and [[Dr. Fuji (the Scientist)]] spend a restless night in a hotel on [[Cinnabar Island]] before departing on the [[Celadon Express]], a Pokémon-based flying service.

#### [[Celadon Express]]

An impressive transportation service utilizing eight massive Pidgeot in V-formation, escorted by a Fearow and Noctowl. Operated by Captains Lin and Reed with assistance from an Ampharos and Magneton, the service provides a "scenic tour" from [[Cinnabar Island]] to Celadon City.

For [[Amber (the Protagonist)]], this journey represents her first real encounter with living Pokémon outside the laboratory setting. Her awe at seeing actual Pidgeot—creatures she previously knew only through games and anime—emphasizes the reality of her new existence. The experience of flight aboard Storm, one of the Pidgeot, proves both exhilarating and overwhelming, eventually lulling her into her first peaceful sleep since her reincarnation.

## Themes and Concepts

### [[Identity and Existence (Amber)]]

A central struggle for [[Amber (the Protagonist)]] revolves around reconciling her past identity as Alexa with her current existence in young Amber's body. She questions whether she has the right to occupy this form, feeling like "an accident of consciousness" rather than a legitimate being. This identity crisis is compounded by the physical disconnect she experiences—adult thoughts trapped in a child's body, with muscle memory and physical sensations that feel foreign and wrong.

### [[Possible Futures (Pokemon: Ambertwo)]]

[[Amber (the Protagonist)]]'s meta-knowledge creates a unique awareness of potential timelines and outcomes. She contemplates various futures for [[Mewtwo (the Legendary Pokémon)]] based on different Pokémon media: capture by [[Team Rocket]] as in the anime, isolation in Cerulean Cave as in the games, or other possibilities. This awareness creates both anxiety and strategic opportunity as she begins navigating this new world.

